# Парсинг и суммаризация данных о лечении аорты с помощью LLM

## Установка зависимостей

In [ ]:
! pip install -qU "langchain-deepseek"
! pip install -qU "langchain-community"
! pip install -q "pypdf"

## Инициализация LLM агента

In [1]:
from langchain_deepseek import ChatDeepSeek


llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [2]:
def get_answer(prompt, **kwargs):
    for k, v in kwargs.items():
        prompt = prompt.replace(k, str(v))

    res = llm.invoke(prompt)
    return res.content

### Пример использования LLM агента

In [3]:
text = "Мне 21 год, мой рост 180 см"

prompt = """
**Задача:**
Выпиши из текста данного ниже только числовые значения через запятую.

**Текст:**
{text}
"""

print(get_answer(prompt, text=text))

21, 180


## Данные

### Метаданные

In [4]:
import json


with open("metadata.json") as f:
    metadata = json.load(f)

In [5]:
# метаданные для первого документа
print(json.dumps(metadata[0], indent=4, ensure_ascii=False))

{
    "filename": "abdominal-aortic-aneurysm.pdf",
    "common_info": "рекомендации по лечению заболеваний брюшной аорты",
    "sections": [
        {
            "title": "Диагностика и лечение",
            "pages": "6-25"
        },
        {
            "title": "Информация об операции",
            "pages": "35-38"
        }
    ]
}


### Разбиение документов на блоки

In [13]:
from tqdm.auto import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter



def get_chunks(filename, page_start=None, page_end=None):
    pages = PyPDFLoader(filename).load_and_split()

    if page_start is None:
        page_start = 0

    if page_end is None:
        page_end = len(pages)

    pages = pages[page_start:page_end]

    full_text = " ".join([page.page_content for page in pages])

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=5000,
        chunk_overlap=1000,
        length_function=lambda text: len(text.split()),
        separators=["\n\n", "\n", " ", ""],
    )

    chunks = text_splitter.create_documents([full_text])

    return chunks

In [14]:
# пример
filename = 'data/' + metadata[0]['filename']
page_diapason = [6, 25]

chunks = get_chunks(
    filename,
    page_start=page_diapason[0],
    page_end=page_diapason[1] + 1,
)
len(chunks)

2

In [15]:
print(chunks[0].page_content)

6
1. Краткая информация 
1.1. Определение 
Аневризма брюшной аорты (АБА) – расширение аорты, в 1,5 раза превышающее ее 
диаметр в нерасширенном участке брюшной аорты, или ее дилатация более 3 см. Можно 
измерить диаметр аорты в любой плоскости, перпендик улярной к ее оси, но чаще измеряется 
переднезадний диаметр, что наиболее наглядно и прос то. В этой связи в большинстве 
скрининговых исследований использовался именно этот размер. 
1.2 Этиология и патогенез 
Наиболее часто аневризмы аорты и периферических арт ерий являются следствием 
деградации медии, вызванной сложными биологическими  механизмами. Традиционно 
считается, что большинство аневризм возникают из-за  дегенеративных атеросклеротических 
изменений, другие данные свидетельствуют о том, что  многие формы аневризм возникают в 
ответ на изменение уровня тканевых металлопротеиназ , что способствует уменьшению 
целостности артериальной стенки. 
К факторам риска АБА относятся:  
• Генетические факторы риска. Риск развития АБА у 

### Сбор информации

In [ ]:
prompt1 = """
**Задача:**
Дан текст из документа, содержащего {text_type}, конкретнее он содержит раздел "{section}".
Нужно из него собрать таблицу, которая будет полезной для написания заключения кардиохирурга.

Критически важны диаметр аорты (например, 4.5 см) и ее локализация (например, восходящая).
Причем важны все параметры аорты, как нормальные, так и отклонения с поставленным диагнозом и лечением.

В ответе должна быть таблица из трех колонок: "диаметр", "диагноз", "лечение и рекомендации".
Таблиц может быть несколько - для каждой локализации аорты.
Если необходимо можно добавить список примечаний после нее.
Не пиши ничего кроме таблиц и списков, а также не придумывай информацию, используй только информацию из текста ниже.

Пример:
"
Восходящая аорта:
| Диаметр (см) | Диагноз | Лечение и рекомендации |
|-------------|---------|--------------|
| **2,0–3,0** | Норма (у взрослых) | Профилактические осмотры раз в 3–5 лет. |
| ... | ... | ... |

Примечания:
- Скорость роста >0,5 см/год – показание к операции даже при меньших размерах.
"

**Текст:**
{text}

**Ответ:**
"""

In [17]:
metadata

[{'filename': 'abdominal-aortic-aneurysm.pdf',
  'common_info': 'рекомендации по лечению заболеваний брюшной аорты',
  'sections': [{'title': 'Диагностика и лечение', 'pages': '6-25'},
   {'title': 'Информация об операции', 'pages': '35-38'}]},
 {'filename': 'Kardiologiya_2018_01_007.pdf',
  'common_info': 'рекомендации по лечению заболеваний аорты',
  'sections': [{'title': 'Диагностика и лечение', 'pages': '5-44'}]},
 {'filename': 'recommendation.pdf',
  'common_info': 'рекомендации по лечению Аневризмы грудной и торакоабдоминальной аорты',
  'sections': [{'title': 'Общая информация', 'pages': '9-14'},
   {'title': 'Диагностика', 'pages': '15-28'},
   {'title': 'Лечение (общее)', 'pages': '29-30'},
   {'title': 'Лечение (АНЕВРИЗМА КОРНЯ И ВОСХОДЯЩЕЙ АОРТЫ)', 'pages': '31-76'},
   {'title': 'Лечение (АНЕВРИЗМА ДУГИ АОРТЫ)', 'pages': '77-101'},
   {'title': 'Лечение (РЕДКИЕ ЗАБОЛЕВАНИЯ ГРУДНОЙ И ТОРАКОАБДОМИНАЛЬНОЙ АОРТЫ)',
    'pages': '102-113'},
   {'title': 'Лечение (АНЕВРИЗМА НИСХ

In [18]:
for file in metadata:
    filename = file['filename']
    text_type = file['common_info']

    for section in file['sections']:
        section_title = section['title']
        page_diapason = list(map(int, section['pages'].split('-')))

        chunks = get_chunks(
            'data/' + filename,
            page_start=page_diapason[0],
            page_end=page_diapason[1] + 1,
        )

        with open(f'extracted/result/{filename}__{section_title}.txt', 'w') as f:
            for chunk in tqdm(chunks, desc=f'{filename} - {section_title}'):
                answer = get_answer(
                    prompt1,
                    text=chunk.page_content,
                    section=section_title,
                    text_type=text_type,
                )

                f.write(answer)
                f.write('\n\n\n\n')
                f.flush()

abdominal-aortic-aneurysm.pdf - Диагностика и лечение:   0%|          | 0/2 [00:00<?, ?it/s]

abdominal-aortic-aneurysm.pdf - Информация об операции:   0%|          | 0/1 [00:00<?, ?it/s]

Kardiologiya_2018_01_007.pdf - Диагностика и лечение:   0%|          | 0/4 [00:00<?, ?it/s]

recommendation.pdf - Общая информация:   0%|          | 0/1 [00:00<?, ?it/s]

recommendation.pdf - Диагностика:   0%|          | 0/1 [00:00<?, ?it/s]

...

recommendation.pdf - Лечение (РЕДКИЕ ЗАБОЛЕВАНИЯ ГРУДНОЙ И ТОРАКОАБДОМИНАЛЬНОЙ АОРТЫ):   0%|          | 0/1 [0…

recommendation.pdf - Лечение (АНЕВРИЗМА НИСХОДЯЩЕЙ ГРУДНОЙ и ТОРАКОАБДОМИНАЛЬНОЙ АОРТЫ):   0%|          | 0/3 …

recommendation2.pdf - Общая информация:   0%|          | 0/1 [00:00<?, ?it/s]

recommendation2.pdf - Диагностика:   0%|          | 0/1 [00:00<?, ?it/s]

recommendation2.pdf - Лечение:   0%|          | 0/10 [00:00<?, ?it/s]

In [20]:
# Пример извлеченного текста
filename = 'recommendation.pdf'
title = 'Лечение (АНЕВРИЗМА ДУГИ АОРТЫ)'

with open(f'extracted/result/{filename}__{title}.txt') as f:
    text = f.read()

print(text)

### Корень аорты:
| Диаметр (см) | Диагноз | Рекомендации |
|-------------|---------|--------------|
| **4,0–4,5** | Генетические синдромы (синдром Марфана, Loeys-Dietz, Ehlers-Danlos) | Хирургическое вмешательство рекомендуется. Для женщин, ожидающих беременности, порог 4,1–4,5 см. |
| **4,5** | Бикуспидальный аортальный клапан, семейная аортопатия | Хирургическое вмешательство рекомендуется. |
| **5,0** | Генетические синдромы (в зависимости от верифицированной патологии) | Хирургическое вмешательство рекомендуется. |
| **5,5** | Дегенеративные аневризмы | Хирургическое вмешательство рекомендуется. |

### Восходящая аорта:
| Диаметр (см) | Диагноз | Рекомендации |
|-------------|---------|--------------|
| **4,2–5,0** | Генетические синдромы (синдром Марфана, Loeys-Dietz, Ehlers-Danlos) | Хирургическое вмешательство рекомендуется. |
| **5,0** | Бикуспидальный аортальный клапан, семейная аортопатия | Хирургическое вмешательство рекомендуется. |
| **5,5** | Дегенеративные аневризмы | Х

### Суммаризация каждого файла

In [22]:
import os


list(sorted(os.listdir('extracted/result/')))

['Kardiologiya_2018_01_007.pdf__Диагностика и лечение.txt',
 'abdominal-aortic-aneurysm.pdf__Диагностика и лечение.txt',
 'abdominal-aortic-aneurysm.pdf__Информация об операции.txt',
 'recommendation.pdf__Диагностика.txt',
 'recommendation.pdf__Лечение (АНЕВРИЗМА ДУГИ АОРТЫ).txt',
 'recommendation.pdf__Лечение (АНЕВРИЗМА КОРНЯ И ВОСХОДЯЩЕЙ АОРТЫ).txt',
 'recommendation.pdf__Лечение (АНЕВРИЗМА НИСХОДЯЩЕЙ ГРУДНОЙ и ТОРАКОАБДОМИНАЛЬНОЙ АОРТЫ).txt',
 'recommendation.pdf__Лечение (РЕДКИЕ ЗАБОЛЕВАНИЯ ГРУДНОЙ И ТОРАКОАБДОМИНАЛЬНОЙ АОРТЫ).txt',
 'recommendation.pdf__Лечение (общее).txt',
 'recommendation.pdf__Общая информация.txt',
 'recommendation2.pdf__Диагностика.txt',
 'recommendation2.pdf__Лечение.txt',
 'recommendation2.pdf__Общая информация.txt']

In [24]:
# big_filename = 'recommendation2.pdf__Лечение.txt' # очень большой файл, нужно раздробить

# with open(f'extracted/result/{big_filename}', 'r') as f:
#     text = f.read()


# blocks = text.split('\n\n\n')
# len(blocks)

In [ ]:
# splits = [
#     '\n\n\n'.join(blocks[10*i:10*(i+1)])
#     for i in range(5)
# ]
# print(len(splits))


# for i in range(5):
#     with open(f'extracted/result/{big_filename[:-4]}_part_{i+1}.txt', 'w') as f:
#         f.write(splits[i])

In [ ]:
files = {
    'всех частей': [
        'Kardiologiya_2018_01_007.pdf__Диагностика и лечение.txt',
    ],

    'восходящей части и корня': [
        'recommendation.pdf__Диагностика.txt',
        'recommendation.pdf__Общая информация.txt',
        'recommendation.pdf__Лечение (АНЕВРИЗМА КОРНЯ И ВОСХОДЯЩЕЙ АОРТЫ).txt',
    ],

    'дуги': [
        'recommendation.pdf__Диагностика.txt',
        'recommendation.pdf__Общая информация.txt',
        'recommendation.pdf__Лечение (АНЕВРИЗМА ДУГИ АОРТЫ).txt'
    ],

    'нисходящей части': [
        'recommendation.pdf__Диагностика.txt',
        'recommendation.pdf__Общая информация.txt',
        'recommendation.pdf__Лечение (АНЕВРИЗМА НИСХОДЯЩЕЙ ГРУДНОЙ и ТОРАКОАБДОМИНАЛЬНОЙ АОРТЫ).txt',
        'recommendation.pdf__Лечение (РЕДКИЕ ЗАБОЛЕВАНИЯ ГРУДНОЙ И ТОРАКОАБДОМИНАЛЬНОЙ АОРТЫ).txt',
    ],

    'брюшной части': [
        'abdominal-aortic-aneurysm.pdf__Диагностика и лечение.txt',
        'abdominal-aortic-aneurysm.pdf__Информация об операции.txt',

        'recommendation2.pdf__Диагностика.txt',
        'recommendation2.pdf__Лечение.txt',
        # 'recommendation2.pdf__Лечение_part_1.txt',
        # 'recommendation2.pdf__Лечение_part_2.txt',
        # 'recommendation2.pdf__Лечение_part_3.txt',
        # 'recommendation2.pdf__Лечение_part_4.txt',
        # 'recommendation2.pdf__Лечение_part_5.txt',
        'recommendation2.pdf__Общая информация.txt',
    ],
}

In [ ]:
prompt2 = """
**Задача:**
Дан текст из документа, содержащего {text_type}, конкретнее он содержит раздел "{section}".
Нужно из него собрать таблицу, которая будет полезной для написания заключения кардиохирурга.

Критически важны диаметр аорты (например, 4.5 см) и ее локализация (например, восходящая).
Причем важны все параметры аорты, как нормальные, так и отклонения с поставленным диагнозом и лечением.

В ответе должна быть таблица из трех колонок: "диаметр", "диагноз", "лечение и рекомендации".
Таблиц может быть несколько - для каждой локализации аорты.
Если необходимо можно добавить список примечаний после нее.
Не пиши ничего кроме таблиц и списков, а также не придумывай информацию, используй только информацию из текста ниже.

Пример:
"
Восходящая аорта:
| Диаметр (см) | Диагноз | Лечение и рекомендации |
|-------------|---------|--------------|
| **2,0–3,0** | Норма (у взрослых) | Профилактические осмотры раз в 3–5 лет. |
| ... | ... | ... |

Примечания:
- Скорость роста >0,5 см/год – показание к операции даже при меньших размерах.
"

**Текст:**
{text}

**Ответ:**
"""

In [30]:
for aorta_type in tqdm(files, desc='FILES SUMMARIZED'):
    for file in tqdm(files[aorta_type], desc=aorta_type):
        with open(f'extracted/result/{file}', 'r') as f:
            answer = get_answer(
                prompt2,
                aorta_type=aorta_type,
                text=f.read(),
            )
        
        with open(f'extracted/summarized/{file}', 'w') as f:
            f.write(answer + '\n')

FILES SUMMARIZED:   0%|          | 0/5 [00:00<?, ?it/s]

всех частей:   0%|          | 0/1 [00:00<?, ?it/s]

восходящей части и корня:   0%|          | 0/3 [00:00<?, ?it/s]

дуги:   0%|          | 0/3 [00:00<?, ?it/s]

нисходящей части:   0%|          | 0/4 [00:00<?, ?it/s]

брюшной части:   0%|          | 0/5 [00:00<?, ?it/s]

### Объединение результатов в один файл

In [ ]:
prompt3 = """
**Задача:**
Даны таблицы из документа, содержащего {text_type}.
Объедини таблицы по смыслу.
Объедини примечания по смыслу.

Критически важны диаметр аорты (например, 4.5 см) и ее локализация (например, восходящая).
Причем важны все параметры аорты, как нормальные, так и отклонения с поставленным диагнозом и лечением.

В ответе должна быть таблица из трех колонок: "диаметр", "диагноз", "лечение и рекомендации".
Таблиц может быть несколько - для каждой локализации аорты.
Список (или списки) примечаний пишутся под таблицами.
Не пиши ничего кроме таблиц и списков, а также не придумывай информацию, используй только информацию из текста ниже.

Пример:
"
Восходящая аорта:
| Диаметр (см) | Диагноз | Лечение и рекомендации |
|-------------|---------|--------------|
| **2,0–3,0** | Норма (у взрослых) | Профилактические осмотры раз в 3–5 лет. |
| ... | ... | ... |

Примечания:
- Скорость роста >0,5 см/год – показание к операции даже при меньших размерах.
"
**Текст:**
{text}

**Ответ:**
"""

In [32]:
for aorta_type in tqdm(files, desc='FILES SUMMARIZED'):
    texts = []
    for file in files[aorta_type]:
        with open(f'extracted/summarized/{file}') as f:
            texts.append(f.read())
    
    text = '\n\n\n'.join(texts)
    answer = get_answer(
        prompt3,
        aorta_type=aorta_type,
        text=text,
    )
        
    with open(f'extracted/final/{aorta_type}.txt', 'w') as f:
        f.write(answer + '\n')

FILES SUMMARIZED:   0%|          | 0/5 [00:00<?, ?it/s]

In [33]:
import os


texts = []
for file in os.listdir('extracted/final/'):
    with open(f'extracted/final/{file}') as f:
        texts.append(f.read())

text = '\n\n\n'.join(texts)
answer = get_answer(
    prompt3,
    aorta_type=aorta_type,
    text=text,
)

with open(f'extracted/summarized.txt', 'w') as f:
    f.write(answer + '\n')